# Trabajo práctico 6 - Transformada de Hough

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

Sobre la imagen de los bloques se pide encontrar las líneas principales que definen a dichos bloques utilizando la transformada de Hough (se pueden realizar procesamientos previos para mejorar la imagen de bordes, ajuste de histograma, filtrado, variantes de binarización, variantes de detección de borde, etc)

**Actividad extra propuesta:** Ajustar rectángulos por transformada de Hough. En los recursos teóricos de la semana "Características locales y Transformada de Hough" hay subido un paper orientativo.


In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 25): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step), rotation=90)
    plt.show()  

A continuación cargamos la imagen a analizar, obtenemos el ancho y el alto y luego la imprimimos

In [ ]:
#Cargamos la imagen a analizar
img = cv.imread('bloques1.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img = cv.medianBlur(img,5) #Para eliminar los posibles ruidos
imgGray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
imgWidth = img.shape[1]
imgHeight = img.shape[0]

print("Img Width: {} \t Img Height: {}".format(imgWidth, imgHeight))
plotter(img, 'Original Image', step=100)
plotter(imgGray, 'Original Image in Gray Scale', grayScale=True, step=100)

In [ ]:
#Ecualizamos el histograma de la imagen que cargamos en escala de grises
imgGrayMod = cv.equalizeHist(imgGray.copy())
plotter(imgGrayMod, 'Equalized', grayScale = True, step=100)

In [ ]:
#Aumentamos el brillo un 30%
imgGrayMod = np.round(np.clip(imgGrayMod + (255*0.30), 0, 255))
plotter(imgGrayMod, grayScale = True, step=100)

## Hough - Líneas

Vamos a generar una máscara, para ello vamos a tomar como muestra cada uno de los bloques con el fin de obtener la mejor máscara posible.

In [ ]:
imgPrueba = img.copy()
# Bloques numerados de arriba hacia abajo y de izq a derecha
#El tercer valor es el ancho del bloque para hacer la mascara
block1 = ((550, 980), (1350, 980), 345)
block2 = ((1500, 1600), (2050, 1600),125)
block3 = ((650, 2400), (1000, 2400), 750)
block4 = ((1300, 2400), (1600, 2400), 700)
block5 = ((1720, 2450), (1850, 2450),350)
block6 = ((1890, 2200), (2250, 2200),800)
blocks = [block1, block2, block3, block4, block5, block6]

# Marcamos una linea roja para ver correctamente sus posiciones
for block in blocks:
    cv.line(imgPrueba, block[0], block[1], (255,0,0), 5)

plotter(imgPrueba, step = 100)

La función createMask es una función auxiliar para poder crear la máscara. Lo que hace es itera cada uno de los bloques que se le pasa por parámetro (variable 'sample') obtiene la media y el desvío de ese bloque y suma esos valores con los valores prevíos. Sería como una especie de promedio de las medias y desvíos de todos los bloques. Finalmente la función devuelve la máscara.

In [ ]:
def createMask(img, samples, lowerMultiplier = 15, upperMultiplier = 6):
    meanColors = 0
    stdColors = 0
    for point1, point2, blockWidth in samples:
        colorMean, colorStd = cv.meanStdDev(img[point1[1]:point1[1] + blockWidth,point1[0]:point2[0], :])
        meanColors += colorMean
        stdColors += colorStd
    meanColors /= len(samples)
    stdColors /= len(samples)
    return cv.inRange(img, meanColors - stdColors * lowerMultiplier,  meanColors + stdColors * upperMultiplier)

In [ ]:
mask = createMask(img, blocks)
imgWithMask = cv.bitwise_and(img, img, mask=mask)
plotter(imgWithMask, 'Image With Mask' ,step = 100)

In [ ]:
imgWithMaskGray = cv.cvtColor(imgWithMask, cv.COLOR_RGB2GRAY)
plotter(imgWithMaskGray,'Gray Image With Mask', grayScale=True, step = 100)

Como podemos observar la máscara que se obtuvo no es lo suficientemente buena, de todas formas 'apaga' muchas cosas que no son necesarias analizar para este caso.
El siguiente bloque de codigo se encuentra marcado como texto, se lo utilizó para encontrar los valores óptimos que luego serán utilizados en la función HoughLinesP

TOP_SLIDER = 100
TOP_SLIDER_MAX = 600
t1 = 10
T1 = 10
edges = 0

def setT1(t1):
    global T1
    T1 = t1
    print(t1)
    if T1 < TOP_SLIDER:
        T1 = 10
    edges = cv.Canny(image, T1, 3*T1) 
    cv.imshow("canny", edges)

image = imgWithMask
edges = cv.Canny(image, T1, 3*T1) 
cv.imshow("canny", edges)
cv.createTrackbar("T1", "canny", t1, TOP_SLIDER_MAX, setT1)

cv.waitKey(0)     
cv.destroyAllWindows()

In [ ]:
edges = cv.Canny(imgWithMask,322,940)
plotter(edges, 'Edges', step=100)

rho_step = 1
theta_step = (np.pi/180)
thresh = 100
lines = cv.HoughLinesP(edges,rho_step,theta_step,thresh, None,200, 600)
imgCopy = img.copy()

# Dibujamos lo resultados
for line in lines:
    x1,y1,x2,y2 = line[0]
    cv.line(imgCopy,(x1,y1),(x2,y2),(255,0,0),5)

plotter(imgCopy,'Result',step=100)

Esta función es similar a la que muestran en los videos de laboratorio, con la diferencia que permite agregar el mínimo y máximo largo de una línea, de esta forma hacemos que sea todo más visible y no haya líneas que atraviesen toda la imagen.

## Conclusiones

Si bien la máscara no fue perfecta permitió encontrar parámetros buenos para luego poder encontrar las líneas de los bloques. La performance de los algoritmos es muy buena y eficiente para encontrar, en este caso, las líneas de todos los bloques.